# This notebook explores the results for the agent based model

In [1]:
import re
import glob
import json
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option( 'display.max_columns', None )
pd.set_option( 'display.max_rows', None )

# Loading localidades GeoJSON for reference

# Loading ABM results

In [173]:
city = 'cartagena'
scenario = 'apertura-50'
scenario_name = 'Apertura 50%'
init_date = pd.to_datetime( '03/06/2020' )

In [174]:
path = './data/agents/{}/{}/*.csv'.format( city, scenario )

In [175]:
files = glob.glob( path )

In [176]:
files.sort()

In [177]:
agents = []
exp = 0

for file in files:
    df = pd.read_csv( file )
    df[ 'Fecha' ] = df[ 'Día' ].apply( lambda x: init_date + pd.DateOffset( days = x ) )
    df[ 'exp' ] = exp
    agents.append( df )
    exp = exp + 1

agents = pd.concat( agents )
del agents[ 'Unnamed: 0' ]

In [178]:
agents.shape

(9600, 43)

In [179]:
agents.dtypes

Susceptibles                                   int64
Total infectados                               int64
Graves                                         int64
Críticos                                       int64
Leves                                          int64
Recuperados                                    int64
Rt                                           float64
Recuento_zonas                                object
0-9                                           object
10-19                                         object
20-39                                         object
40-59                                         object
>60                                           object
En_cuarentena                                 object
Restringido_movilidad                         object
Vivos                                          int64
Día                                            int64
Contactos_prom_trabajo                       float64
Contactos_prom_transporte                    f

In [180]:
agents.tail()

,Susceptibles,Total infectados,Graves,Críticos,Leves,Recuperados,Rt,Recuento_zonas,0-9,10-19,20-39,40-59,>60,En_cuarentena,Restringido_movilidad,Vivos,Día,Contactos_prom_trabajo,Contactos_prom_transporte,Contactos_prom_casa,Contactos_prom_trabajo_conm,Contactos_prom_transporte_conm,Contactos_prom_casa_conm,Nuevos_infectados,Detectados,En_testing,En_cama,En_UCI,Detectados_por_intervencion,#Intervenidos,porcentaje_edad_activa_por_zona,porcentaje_edad_activa,tasa_letalidad_por_zona,tasa_letalidad,total_muertos,total_muertos_por_zona,total_muertos_edad_activa,total_muertos_edad_activa_por_zona,zonas intervenidas,Graves_efectivos,Criticos_efectivos,Fecha,exp
295,42172,746,17,3,726,11945,0.512064,"{'COMUNA 11': [2492, 0, 0, 4, 2, 0, 1, 0, 1, 0...","[6676, 0, 0, 129, 11, 0, 0, 0, 0, 0, 0, 0, 192...","[7087, 0, 0, 123, 6, 0, 0, 0, 0, 0, 0, 0, 1905...","[13883, 0, 0, 238, 10, 0, 3, 0, 3, 0, 0, 0, 40...","[9387, 0, 0, 146, 4, 0, 9, 0, 6, 1, 0, 1, 2757...","[5139, 0, 0, 90, 4, 0, 5, 0, 4, 2, 0, 2, 1322,...","[0, 0, 2, 0, 0]","[18157, 302, 3, 1, 5006]",54863,295,0.337368,0.736453,3.889853,0.589609,1.287080,3.873598,35,312,4,74,4,0,0,"{'COMUNA 8': 0.006418485237483954, 'COMUNA 1':...",0.003205,"{'COMUNA 8': 0.18421052631578946, 'COMUNA 1': ...",0.147541,54,"{'COMUNA 8': 7, 'COMUNA 1': 2, 'COMUNA 4': 1, ...",11,"{'COMUNA 8': 1, 'COMUNA 1': 0, 'COMUNA 4': 1, ...",{},8,2,2020-12-26,31
296,42145,728,16,3,709,11990,0.508242,"{'COMUNA 11': [2491, 0, 0, 4, 1, 0, 1, 0, 1, 0...","[6669, 0, 0, 128, 5, 0, 0, 0, 0, 0, 0, 0, 1931...","[7086, 0, 0, 119, 3, 0, 0, 0, 0, 0, 0, 0, 1910...","[13874, 0, 0, 226, 9, 0, 3, 0, 3, 0, 0, 0, 405...","[9378, 0, 0, 146, 5, 0, 8, 0, 5, 1, 0, 1, 2767...","[5138, 0, 0, 90, 6, 0, 5, 0, 4, 2, 0, 2, 1323,...","[0, 0, 0, 0, 0]","[18828, 325, 6, 0, 5318]",54863,296,0.313672,0.686528,3.892277,0.566346,1.239551,3.876588,28,312,4,74,4,0,0,"{'COMUNA 8': 0.006418485237483954, 'COMUNA 1':...",0.003205,"{'COMUNA 8': 0.18421052631578946, 'COMUNA 1': ...",0.147541,54,"{'COMUNA 8': 7, 'COMUNA 1': 2, 'COMUNA 4': 1, ...",11,"{'COMUNA 8': 1, 'COMUNA 1': 0, 'COMUNA 4': 1, ...",{},8,2,2020-12-27,31
297,42125,705,16,3,686,12032,0.496454,"{'COMUNA 11': [2491, 0, 0, 4, 0, 0, 1, 0, 1, 0...","[6663, 0, 0, 125, 7, 0, 0, 0, 0, 0, 0, 0, 1940...","[7086, 0, 0, 111, 1, 0, 0, 0, 0, 0, 0, 0, 1918...","[13867, 0, 0, 220, 9, 0, 3, 0, 2, 0, 0, 0, 407...","[9375, 0, 0, 141, 6, 0, 8, 0, 5, 1, 0, 1, 2775...","[5134, 0, 0, 89, 0, 0, 5, 0, 4, 2, 0, 2, 1327,...","[0, 0, 0, 0, 0]","[17816, 296, 2, 1, 4965]",54862,297,0.343717,0.758649,3.890762,0.593323,1.309578,3.876439,23,312,5,74,4,0,0,"{'COMUNA 8': 0.006418485237483954, 'COMUNA 1':...",0.003205,"{'COMUNA 8': 0.18421052631578946, 'COMUNA 1': ...",0.149864,55,"{'COMUNA 8': 7, 'COMUNA 1': 2, 'COMUNA 4': 1, ...",11,"{'COMUNA 8': 1, 'COMUNA 1': 0, 'COMUNA 4': 1, ...",{},9,2,2020-12-28,31
298,42095,679,15,2,662,12088,0.474227,"{'COMUNA 11': [2491, 0, 0, 4, 0, 0, 1, 0, 1, 0...","[6657, 0, 0, 128, 5, 0, 0, 0, 0, 0, 0, 0, 1943...","[7084, 0, 0, 104, 1, 0, 0, 0, 0, 0, 0, 0, 1927...","[13858, 0, 0, 202, 7, 0, 3, 0, 2, 0, 0, 0, 409...","[9364, 0, 0, 143, 6, 0, 8, 0, 6, 1, 0, 1, 2784...","[5132, 0, 0, 85, 4, 0, 4, 0, 3, 1, 0, 1, 1335,...","[0, 0, 1, 0, 0]","[17749, 285, 1, 0, 5002]",54862,298,0.347891,0.754092,3.889596,0.599736,1.299994,3.870287,23,313,4,74,4,0,0,"{'COMUNA 8': 0.006418485237483954, 'COMUNA 1':...",0.003223,"{'COMUNA 8': 0.18421052631578946, 'COMUNA 1': ...",0.149457,55,"{'COMUNA 8': 7, 'COMUNA 1': 2, 'COMUNA 4': 1, ...",11,"{'COMUNA 8': 1, 'COMUNA 1': 0, 'COMUNA 4': 1, ...",{},9,2,2020-12-29,31
299,42055,684,14,2,668,12123,0.475146,"{'COMUNA 11': [2490, 0, 0, 5, 1, 0, 1, 0, 1, 0...","[6652, 0, 0, 123, 6, 0, 0, 0, 0, 0, 0, 0, 1953...","[7077, 0, 0, 107, 3, 0, 0, 0, 0, 0, 0, 0, 1931...","[13846, 0, 0, 204, 11, 0, 3, 0, 2, 0, 0, 0, 41...","[9353, 0, 0, 147, 10, 0, 7, 0, 7, 1, 0, 1, 279...","[5127, 0, 0, 87, 6, 0, 4, 0, 4, 1, 0, 1, 1338,...","[0, 0, 3, 0, 0]","[7880, 117, 5, 1, 2294]",54862,299,0.785626,1.687

In [181]:
def sum_quarantine( x ):
    x = json.loads( x )
    people_in_q = x[ 0 ] + x[ 1 ] + x[ 2 ] + x[ 3 ] + x[ 4 ]
    return people_in_q

In [182]:
agents_norm = []

for exp in agents[ 'exp' ].unique():
    df = agents.loc[ agents[ 'exp' ] == exp ]
    total_people = df[ 'Vivos' ].values[ 0 ]
    df[ 'Graves' ] = df[ 'Graves' ] / total_people
    df[ 'Críticos' ] = df[ 'Críticos' ] / total_people
    df[ 'Fallecidos' ] = ( total_people - df[ 'Vivos' ] ) / total_people
    df[ 'R0' ] = ( df[ 'Nuevos_infectados' ].cumsum() / total_people )
    df[ 'Cuarentena' ] = df[ 'En_cuarentena' ].apply( lambda x: sum_quarantine( x ) ) / total_people
    agents_norm.append( df[ [ 'exp', 'Fecha', 'Graves', 'Críticos', 'Fallecidos', 'R0', 'Rt', 'Cuarentena' ] ] )

agents_norm = pd.concat( agents_norm )

/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [183]:
agents_norm.tail()

,exp,Fecha,Graves,Críticos,Fallecidos,R0,Rt,Cuarentena
295,31,2020-12-26,0.000310,0.000055,0.000983,0.230839,0.512064,0.000036
296,31,2020-12-27,0.000291,0.000055,0.000983,0.231349,0.508242,0.000000
297,31,2020-12-28,0.000291,0.000055,0.001002,0.231768,0.496454,0.000000
298,31,2020-12-29,0.000273,0.000036,0.001002,0.232187,0.474227,0.000018
299,31,2020-12-30,0.000255,0.000036,0.001002,0.232842,0.475146,0.000055


In [184]:
var = 'Críticos'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}-{}-{}.csv'.format( city, city, scenario, 'ucis' ), index = False )

var = 'Fallecidos'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}-{}-{}.csv'.format( city, city, scenario, 'deaths' ), index = False )

var = 'R0'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}-{}-{}.csv'.format( city, city, scenario, 'r0' ), index = False )

var = 'Rt'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}-{}-{}.csv'.format( city, city, scenario, 'rt' ), index = False )

var = 'Cuarentena'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}-{}-{}.csv'.format( city, city, scenario, 'quarantine' ), index = False )

In [ ]:
def ci( x ):
    try:
        return 1.96 * np.std( x ) / np.sqrt( len( x ) )
    except ZeroDivisionError:
        return 0

agents_norm_grouped = agents_norm.groupby( 'Fecha' ).agg( { 'Graves': [ 'mean', ci ], 'Críticos': [ 'mean', ci ], 'Fallecidos': [ 'mean', ci ], 'R0': [ 'mean', ci ], 'Rt': [ 'mean', ci ], 'Cuarentena': [ 'mean' ] } )
agents_norm_grouped.columns = [ '_'.join( col ).strip() for col in agents_norm_grouped.columns.values ]

In [ ]:
agents_norm_grouped.tail()

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Graves_mean' ], label = 'Graves', color = '#317372' )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Graves_mean' ] - agents_norm_grouped[ 'Graves_ci' ] ), ( agents_norm_grouped[ 'Graves_mean' ] + agents_norm_grouped[ 'Graves_ci' ] ), color = '#317372', alpha = .1 )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Críticos_mean' ], label = 'Críticos', color = '#f98a4b' )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Críticos_mean' ] - agents_norm_grouped[ 'Críticos_ci' ] ), ( agents_norm_grouped[ 'Críticos_mean' ] + agents_norm_grouped[ 'Críticos_ci' ] ), color = '#f98a4b', alpha = .1 )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Fallecidos_mean' ], label = 'Fallecidos', color = '#f64438' )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Fallecidos_mean' ] - agents_norm_grouped[ 'Fallecidos_ci' ] ), ( agents_norm_grouped[ 'Fallecidos_mean' ] + agents_norm_grouped[ 'Fallecidos_ci' ] ), color = '#f64438', alpha = .1 )

plt.title( 'Graves / Críticos / Fallecidos' )
plt.xlabel( 'Fecha' )
plt.ylabel( 'Casos' )
plt.grid( True )
plt.legend()

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'R0_mean' ] )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'R0_mean' ] - agents_norm_grouped[ 'R0_ci' ] ), ( agents_norm_grouped[ 'R0_mean' ] + agents_norm_grouped[ 'R0_ci' ] ), alpha = .1 )

plt.title( 'Tasa de ataque' )
plt.xlabel( 'Fecha' )
plt.ylabel( 'R0' )
plt.grid( True )

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Rt_mean' ] )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Rt_mean' ] - agents_norm_grouped[ 'Rt_ci' ] ), ( agents_norm_grouped[ 'Rt_mean' ] + agents_norm_grouped[ 'Rt_ci' ] ), alpha = .1 )

plt.title( 'Tasa de reproducción afectiva' )
plt.xlabel( 'Fecha' )
plt.ylabel( 'Rt' )
plt.grid( True )

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Cuarentena_mean' ] )

plt.title( 'Cuarentena' )
plt.xlabel( 'Fecha' )
plt.ylabel( '% cuarentena' )
plt.grid( True )

In [ ]:
agents_norm.tail()

In [ ]:
agents_norm[ [ 'exp', 'Fecha', 'Graves', 'Críticos', 'Fallecidos', 'R0', 'Rt', 'Cuarentena' ] ].to_csv( './data/agents/clean/{}-{}.csv'.format( city, scenario ), index = False )